Initial experiments with learning pytorch
Starting with Atticus' hierarchical equality code!
Stolen from https://github.com/atticusg/InterchangeInterventions/blob/main/iit_equality.ipynb

In [1]:
import torch
from torch import nn
import random
import utilities
import copy
import itertools
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from LIM_deep_neural_classifier import LIMDeepNeuralClassifier
import dataset_equality
from trainer import LIMTrainer


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
utilities.fix_random_seeds()

The intervention graph:

In [3]:
def compute_A(ex, intervention):
    graph = {}
    for i, obj in enumerate(ex):
        graph["input" + str(i+1)] = obj
    if "V1" in intervention:
        graph["V1"] = intervention["V1"]
    else:
        graph["V1"] = graph["input1"] == graph["input2"]
    if "V2" in intervention:
        graph["V2"] = intervention["V2"]
    else:
        graph["V2"] = graph["input3"] == graph["input4"]
    graph["output"] = graph["V1"] == graph["V2"]
    return graph

In [4]:
compute_A(
    ("pentagon", "pentagon", "triangle", "square"), 
    intervention={})

compute_A(
    ("square", "pentagon", "triangle", "triangle"), 
    intervention={"V1": True})

{'input1': 'square',
 'input2': 'pentagon',
 'input3': 'triangle',
 'input4': 'triangle',
 'V1': True,
 'V2': True,
 'output': True}

In [5]:
def compute_interchange_A(base, source, variable):
    # Run the algorithm on `source`:
    src_output = compute_A(source, intervention={})
    # Get the source value for `variable`:
    val = src_output[variable]
    # Process `base` with the intervention setting `variable`
    # to the value it had in `source`:        
    return compute_A(base, intervention={variable: val})

In [6]:
compute_interchange_A(
    base=("pentagon", "pentagon", "triangle", "square"),    # base: T F ==> F
    source=("square", "pentagon", "triangle", "triangle"),  # source: F T ==> F
    variable="V1")

{'input1': 'pentagon',
 'input2': 'pentagon',
 'input3': 'triangle',
 'input4': 'square',
 'V1': False,
 'V2': False,
 'output': True}

In [7]:
embedding_dim = 4

n_examples = 100000

X_train, X_test, y_train, y_test, test_dataset = dataset_equality.get_equality_dataset(
    embedding_dim, n_examples)

In [8]:
X_train[0], y_train[0]

(tensor([ 0.2756, -0.4622,  0.3554,  0.0986,  0.2756, -0.4622,  0.3554,  0.0986,
          0.3294,  0.4381, -0.2060, -0.2393,  0.4394,  0.2022, -0.3943, -0.1066],
        dtype=torch.float64),
 0)

In [9]:
left = torch.equal(
    X_train[0][: embedding_dim],
    X_train[0][embedding_dim: embedding_dim*2])

left

True

In [10]:
right = torch.equal(
    X_train[0][embedding_dim*2: embedding_dim*3],
    X_train[0][embedding_dim*3: ])

right

False

In [11]:
int(left == right)

0

In [12]:

LIM = LIMDeepNeuralClassifier(
    hidden_dim=embedding_dim*4, 
    hidden_activation=torch.nn.ReLU(), 
    num_layers=3,
    input_dim=embedding_dim*4,
    n_classes=2
    )
LIM_trainer = LIMTrainer(
    LIM,
    warm_start=True,
    max_iter=10,
    batch_size=64,
    n_iter_no_change=10000,
    shuffle_train=False,
    eta=0.001)

Let's rip out and replace the above, and keep it much simpler, since this doesn't seem to be working anyway.
We'll start by eliminating the details of the rotatey stuff; since we'll want to do that in our own way later, anyway

In [13]:
#Making a 3-layer relu to do the task:
class ThreeLayerReLU(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(ThreeLayerReLU, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        x = self.layer3(x)
        return x

Now instantiating the network

In [14]:
net = ThreeLayerReLU(16, 16, 16, 1)
X_train = X_train.float()
net(X_train)

tensor([[0.1697],
        [0.1398],
        [0.1607],
        ...,
        [0.1744],
        [0.1382],
        [0.1990]], grad_fn=<AddmmBackward0>)

In [15]:
X_train.size()

torch.Size([100000, 16])

In [16]:
import torch.optim as optim

def train(net, X_train, Y_train, epochs=100):
    # Define a loss function and an optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(net.parameters())

    for epoch in range(epochs):  
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(X_train)
        # Compute the loss
        loss = loss_function(outputs, Y_train)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


In [17]:
#this is not working right now, not sure why, but I figured out enough to keep working on atticus' code directly
#train(net, X_train, y_train)

AttributeError: 'tuple' object has no attribute 'size'

Oh, hey, I figured out the problem! Running everything ab

In [18]:
LIM_trainer.fit(X_train, y_train)


Finished epoch 10 of 10; error is 17.836314604996005

LIMTrainer(
	batch_size=64,
	max_iter=10,
	eta=0.001,
	optimizer_class=<class 'torch.optim.adam.Adam'>,
	l2_strength=0,
	gradient_accumulation_steps=1,
	max_grad_norm=None,
	validation_fraction=0.1,
	early_stopping=False,
	n_iter_no_change=10000,
	warm_start=True,
	tol=1e-05)

In [24]:
train_preds = LIM_trainer.predict(X_train, device="cpu")
train_preds = train_preds.cpu().numpy().astype('int64')

classification_report(y_train, train_preds)
#print("Train Results")
#print(classification_report(y_train, train_preds))

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00     50000\n           1       1.00      0.99      1.00     50000\n\n    accuracy                           1.00    100000\n   macro avg       1.00      1.00      1.00    100000\nweighted avg       1.00      1.00      1.00    100000\n'

In [27]:
print("Test Results")

test_preds = LIM_trainer.predict(X_test, device="cpu")
test_preds = test_preds.cpu().numpy().astype('int64')

print(classification_report(y_test, test_preds))

Test Results
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     50000
           1       1.00      0.99      1.00     50000

    accuracy                           1.00    100000
   macro avg       1.00      1.00      1.00    100000
weighted avg       1.00      1.00      1.00    100000



In [28]:
a = X_train[0][: embedding_dim]
b = X_train[1][: embedding_dim]
c = X_train[2][: embedding_dim]

X_same_different = torch.cat((a, a, b, c)).unsqueeze(0)

X_different_same = torch.cat((a, b, c, c)).unsqueeze(0)

In [29]:
zeroing_get_coord = {
    "layer": 1,
    "start": 0, 
    "end": 
    embedding_dim*4
}

In [37]:
zeroing_intervention = [{
    "layer": 1,
    "start": 0,  
    "end": embedding_dim, 
    "intervention": torch.zeros((1,embedding_dim))
}]

In [38]:
LIM_trainer.model.retrieve_activations(X_same_different, zeroing_get_coord, None)


tensor([[0.5062, 0.0083, 0.0000, 0.0974, 1.2192, 0.7053, 0.7218, 0.1774, 0.6769,
         0.0000, 0.0206, 0.0000, 0.7264, 0.0000, 0.0000, 0.7720]],
       device='cuda:0', grad_fn=<SliceBackward0>)

weirdly this doesn't seem to do anything? lol i wonder if we should fix that!

In [40]:
LIM_trainer.model

LIMDeepNeuralClassifier(
  (hidden_activation): ReLU()
  (loss): CrossEntropyLoss()
  (model_layers): ModuleList(
    (0): ActivationLayer(
      (linear): Linear(in_features=16, out_features=16, bias=True)
      (activation): ReLU()
    )
    (1): ActivationLayer(
      (linear): Linear(in_features=16, out_features=16, bias=True)
      (activation): ReLU()
    )
    (2): ActivationLayer(
      (linear): Linear(in_features=16, out_features=16, bias=True)
      (activation): ReLU()
    )
    (3): Linear(in_features=16, out_features=2, bias=True)
  )
  (analysis_model): Sequential(
    (0): ActivationLayer(
      (linear): Linear(in_features=16, out_features=16, bias=True)
      (activation): ReLU()
    )
    (1): ParametrizedLinearLayer(
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): _Orthogonal()
        )
      )
    )
    (2): InverseLinearLayer(
      (lin_layer): ParametrizedLinearLayer(
        (parametrizations): ModuleDict(
          

In [39]:
LIM_trainer.model.retrieve_activations(X_same_different, zeroing_get_coord, zeroing_intervention)


tensor([[0.5062, 0.0083, 0.0000, 0.0974, 1.2192, 0.7053, 0.7218, 0.1774, 0.6769,
         0.0000, 0.0206, 0.0000, 0.7264, 0.0000, 0.0000, 0.7720]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [36]:
zeroing_output_coord = {
    "layer": 3, 
    "start": 0, 
    "end": 2}

In [41]:
LIM_trainer.model.retrieve_activations(X_same_different, zeroing_output_coord, sets=None)


tensor([[ 4.5777, -2.5793]], device='cuda:0', grad_fn=<SliceBackward0>)

In [42]:
LIM_trainer.model.retrieve_activations(X_same_different, zeroing_output_coord, zeroing_intervention)


tensor([[ 3.7685, -1.8232]], device='cuda:0', grad_fn=<SliceBackward0>)

In [43]:
ii_coord = {"layer": 1, "start": 0, "end": embedding_dim}


In [44]:
intervention_get = LIM_trainer.model.retrieve_activations(X_different_same, ii_coord, None)

intervention_get

tensor([[0.5422, 0.0682, 0.0000, 0.6212]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [45]:
ii_set = [{
    "layer": 1, 
    "start": 0, 
    "end": embedding_dim, 
    "intervention": intervention_get}]

In [46]:
LIM_trainer.model.retrieve_activations(X_same_different, ii_coord, None)


tensor([[0.5062, 0.0083, 0.0000, 0.0974]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [47]:
LIM_trainer.model.retrieve_activations(X_same_different, ii_coord, ii_set)


tensor([[0.5062, 0.0083, 0.0000, 0.0974]], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [48]:
ii_output_coord = {"layer": 3, "start": 0, "end": 2}


In [49]:
LIM_trainer.model.retrieve_activations(X_same_different, ii_output_coord, None)


tensor([[ 4.5777, -2.5793]], device='cuda:0', grad_fn=<SliceBackward0>)

In [50]:
LIM_trainer.model.retrieve_activations(X_same_different, ii_output_coord, ii_set)


tensor([[ 3.8578, -1.4604]], device='cuda:0', grad_fn=<SliceBackward0>)

In [51]:

alignment = {
    "V1": {"layer": 1, "start": 0, "end": embedding_dim}, 
    "V2": {"layer": 1, "start": embedding_dim, "end": embedding_dim*2}}

In [52]:
def interchange_intervention(model, base, source, get_coord, output_coord):
    intervention = model.retrieve_activations(source, get_coord, None)
    get_coord["intervention"] = intervention
    return model.retrieve_activations(base, output_coord, [get_coord])

In [53]:
output_coord = {"layer": 3, "start": 0, "end": 2}


In [54]:
interchange_intervention(
    LIM_trainer.model, 
    base=X_same_different, 
    source=X_different_same, 
    get_coord=ii_coord, 
    output_coord=output_coord)

tensor([[ 3.8578, -1.4604]], device='cuda:0', grad_fn=<SliceBackward0>)

In [55]:
def convert_input(tensor, embedding_dim):
    return [tuple(tensor[0, embedding_dim*k:embedding_dim*(k+1)].flatten().tolist()) 
            for k in range(4)]

In [56]:
compute_A(convert_input(X_same_different, embedding_dim), {})['output']


False

In [57]:
compute_A(convert_input(X_different_same, embedding_dim), {})['output']


False

In [58]:
compute_interchange_A(
    convert_input(X_different_same, embedding_dim),
    convert_input(X_same_different, embedding_dim),
    variable="V1")['output']    

True

In [59]:
def ii_evaluation(X_assess, model, variable, output_coord):
    labels = []
    predictions = []
    for base, source in itertools.product(X_assess, repeat=2):
        base = base.unsqueeze(0)
        source = source.unsqueeze(0)
        # Run the high-level model with the intervention:
        algorithm_output = compute_interchange_A(
            convert_input(base, embedding_dim), 
            convert_input(source, embedding_dim), 
            variable)
        # Get the high-level model's label:
        labels.append(int(algorithm_output["output"]))
        # Run the neural model with the intervention:
        network_output = interchange_intervention(
            model, 
            base,
            source,
            alignment[variable],
            output_coord)
        # Get the neural model's prediction with the intervention:
        pred = network_output.argmax(axis=1)
        predictions.append(int(pred))
    return labels, predictions

In [60]:
print(classification_report(*ii_evaluation(X_test[: 100], LIM_trainer.model, "V1", output_coord)))


              precision    recall  f1-score   support

           0       0.65      0.64      0.64      5048
           1       0.64      0.64      0.64      4952

    accuracy                           0.64     10000
   macro avg       0.64      0.64      0.64     10000
weighted avg       0.64      0.64      0.64     10000



In [61]:
print(classification_report(*ii_evaluation(X_test[: 100], LIM_trainer.model, "V2", output_coord)))


              precision    recall  f1-score   support

           0       0.56      0.72      0.63      5048
           1       0.59      0.41      0.48      4952

    accuracy                           0.57     10000
   macro avg       0.57      0.57      0.56     10000
weighted avg       0.57      0.57      0.56     10000



It's not basis-aligned, I guess?
task for tomorrow: implement this but with zen's rotational alignment!